# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [2]:
table = pd.read_csv('data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [3]:
table.columns = [i.strip() for i in table.columns]

Let's evaluate the dataset by looking at the `head` function.

In [4]:
table.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [5]:
table.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [6]:
table = table.set_index('Serial No.',drop=False)

Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [7]:
print("Length of table:", table['Serial No.'].size)
print("Unique GRE/CGPA:", table.groupby(['GRE Score', 'CGPA']).ngroups)

Length of table: 385
Unique GRE/CGPA: 385


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [8]:
table = table.set_index(['GRE Score', 'CGPA'])
table.reset_index(inplace=True)

# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [9]:
table.loc[(table['CGPA'] >9) & (table['Research'] == 1)].head(10)

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
4,330,9.34,5,115,5,4.5,3.0,1,0.90
10,328,9.10,11,112,4,4.0,4.5,1,0.78
19,328,9.50,20,116,5,5.0,5.0,1,0.94
20,334,9.70,21,119,5,5.0,4.5,1,0.95
21,336,9.80,22,119,5,4.0,3.5,1,0.97
28,338,9.40,29,118,4,3.0,4.5,1,0.91
29,340,9.60,30,114,5,4.0,4.0,1,0.90
30,331,9.80,31,112,5,4.0,5.0,1,0.94
31,320,9.20,32,110,5,5.0,5.0,1,0.88


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [10]:
new_table = table.loc[(table['CGPA'] >9) & (table['SOP'] < 3.5)]
new_table['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [11]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return col - col.mean()/col.std()


Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [12]:
table['CGPA_std'] = standardize(table['CGPA'])
table['GRE_std'] = standardize(table['GRE Score'])
table['LOR_std'] = standardize(table['LOR'])
table.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.621747,309.598899,0.674568
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.271747,288.598899,-0.325432
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.601747,294.598899,-1.325432
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.061747,286.598899,-0.825432
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.931747,302.598899,-0.825432


We will generate the decision choice at random using the code below. Please run the cell.

In [13]:
# Libraries
from random import choices

In [14]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=table.shape[0])
decision_choice

['CGPA_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [15]:
table['deciding_columm'] =table.lookup(table.index, decision_choice)
table.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_columm
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.621747,309.598899,0.674568,-4.621747
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.271747,288.598899,-0.325432,288.598899
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.601747,294.598899,-1.325432,-5.601747
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.061747,286.598899,-0.825432,-6.061747
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.931747,302.598899,-0.825432,-0.825432


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [20]:
table['decision'] = np.where(table['deciding_columm'] >= 0.8,1,0) 
table.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_columm,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,-4.621747,309.598899,0.674568,-4.621747,0
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-6.271747,288.598899,-0.325432,288.598899,1
2,322,8.67,3,110,3,3.5,2.5,1,0.80,-5.601747,294.598899,-1.325432,-5.601747,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-6.061747,286.598899,-0.825432,-6.061747,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,-4.931747,302.598899,-0.825432,-0.825432,0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [23]:
print("Number of applicants to be accepted onto the program:", table['decision'].sum())

Number of applicants to be accepted onto the program: 143


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [ ]:
table.columns = table.columns.str.replace('[^a-zA-Z0-9]', '')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
def add_ten(x):
    return x + 10
adjusted_gre = pd.cut(np.where(table['UniversityRating'] >= 4, 
                                         table['GREScore'].apply(add_ten), table['GREScore']), bins=4)
adjusted_gre